#  Tut 3a- Function calling with the Gemini API-

In this notebook, you'll learn the Gemini API to use functions (like tools) to make a chat that can talk to a database. This lets you ask questions and get answers from your data through chat, without needing to know database commands. We'll build a simple version to show how it works.

In [1]:
!pip install -U -q "google-genai==1.7.0"
from google import genai
from google.genai import types

genai.__version__

from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.3 MB/s eta 0:00:00


### **Retry if Needed:**

* We're setting up a rule that says, "If the Gemini API gives us an error because it's too busy (like a traffic jam), try sending the request again."
* This helps make sure our program keeps working even if the API is having a temporary problem.
* We're essentially telling it to be patient and try again.

In [4]:
from google.api_core import retry

is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

if not hasattr(genai.models.Models.generate_content, '__wrapped__'):
  genai.models.Models.generate_content = retry.Retry(
      predicate=is_retriable)(genai.models.Models.generate_content)

## **Make a Small Database:** We'll create a simple database.

##  **Add Some Data:** We'll put some fake data into it so we have something to ask questions about.

##  **Use SQL Commands:** We'll use special commands to talk to the database and create it.

In [10]:
%load_ext sql
%sql sqlite:///sample.db



The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [23]:
%%sql
CREATE TABLE IF NOT EXISTS products (
    product_id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name VARCHAR(255) NOT NULL,
    price DECIMAL(10, 2) NOT NULL
);

 * sqlite:///sample.db
Done.


[]

In [24]:
%%sql
CREATE TABLE IF NOT EXISTS staff (
  	staff_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	first_name VARCHAR(255) NOT NULL,
  	last_name VARCHAR(255) NOT NULL
  );

 * sqlite:///sample.db
Done.


[]

In [30]:
%%sql
CREATE TABLE IF NOT EXISTS orders (
  	order_id INTEGER PRIMARY KEY AUTOINCREMENT,
  	customer_name VARCHAR(255) NOT NULL,
  	staff_id INTEGER NOT NULL,
  	product_id INTEGER NOT NULL,
  	FOREIGN KEY (staff_id) REFERENCES staff (staff_id),
  	FOREIGN KEY (product_id) REFERENCES products (product_id)
  );


%%sql
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);

%%sql
INSERT INTO staff (first_name, last_name) VALUES
  	('Alice', 'Smith'),
  	('Bob', 'Johnson'),
  	('Charlie', 'Williams
    
%%sql
INSERT INTO orders (customer_name, staff_id, product_id) VALUES
  	('David Lee', 1, 1),
  	('Emily Chen', 2, 2),
  	('Frank Brown', 1, 3);

 * sqlite:///sample.db
Done.
(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql
INSERT INTO products (product_name, price) VALUES
  	('Laptop', 799.99),
  	('Keyboard', 129.99),
  	('Mouse', 29.99);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [31]:
import sqlite3

db_file = "sample.db"
db_conn = sqlite3.connect(db_file)

#### The first function will list all tables available in the database. Define it, and test it out to ensure it works.

In [36]:
def list_tables() -> list[str]:
    """Retrieve the names of all tables in the database."""
    # Include print logging statements so you can see when functions are being called.
    print(' - DB CALL: list_tables()')

    cursor = db_conn.cursor()

    # Fetch the table names.
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

    tables = cursor.fetchall()
    return [t[0] for t in tables]


list_tables()

 - DB CALL: list_tables()


['products', 'sqlite_sequence', 'staff', 'orders']

In [32]:
def describe_table(table_name: str) -> list[tuple[str, str]]:
    """Look up the table schema.

    Returns:
      List of columns, where each entry is a tuple of (column, type).
    """
    print(f' - DB CALL: describe_table({table_name})')

    cursor = db_conn.cursor()

    cursor.execute(f"PRAGMA table_info({table_name});")

    schema = cursor.fetchall()
    # [column index, column name, column type, ...]
    return [(col[1], col[2]) for col in schema]


describe_table("products")

 - DB CALL: describe_table(products)


[('product_id', 'INTEGER'),
 ('product_name', 'VARCHAR(255)'),
 ('price', 'DECIMAL(10, 2)')]

In [34]:
def execute_query(sql: str) -> list[list[str]]:
    """Execute an SQL statement, returning the results."""
    print(f' - DB CALL: execute_query({sql})')

    cursor = db_conn.cursor()

    cursor.execute(sql)
    return cursor.fetchall()


execute_query("select * from products")

 - DB CALL: execute_query(select * from products)


[(1, 'Laptop', 799.99), (2, 'Keyboard', 129.99), (3, 'Mouse', 29.99)]

**Use Gemini with Tools:** We're going to make Gemini use our database tools (the functions we defined).

**Chat with Functions:**
* We'll start a chat with Gemini.
* We'll give Gemini the descriptions of our tools.
* Instead of just giving text answers, Gemini might ask to use a tool (a function call).

**Automatic Tools:**
* We'll set it up so Gemini automatically uses the tools when it needs to.
* This lets Gemini get info from the database without us having to tell it every step.

**How it Works:**
* We tell Gemini about the database and the tools.
* We ask Gemini a question.
* Gemini decides if it needs to use a tool (like asking the database a question).
* Gemini then gets the answer and tells us.
* We can keep chatting.

**In simple terms:** We're giving Gemini special tools to talk to our database, and letting it use them automatically to answer our questions.

In [37]:
# These are the Python functions defined above.
db_tools = [list_tables, describe_table, execute_query]

instruction = """You are a helpful chatbot that can interact with an SQL database
for a computer store. You will take the users questions and turn them into SQL
queries using the tools available. Once you have the information you need, you will
answer the user's question using the data returned.

Use list_tables to see what tables are present, describe_table to understand the
schema, and execute_query to issue an SQL SELECT query."""

client = genai.Client(api_key=GOOGLE_API_KEY)

# Start a chat with automatic function calling enabled.
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

In [38]:
resp = chat.send_message("What is the cheapest product?")
print(f"\n{resp.text}")

 - DB CALL: execute_query(SELECT ProductName, Price FROM Products ORDER BY Price ASC LIMIT 1;)
 - DB CALL: describe_table(Products)
 - DB CALL: execute_query(SELECT product_name, price FROM Products ORDER BY price ASC LIMIT 1;)

The cheapest product is the Mouse, which costs $29.99.



In [39]:
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction=instruction,
        tools=db_tools,
    ),
)

response = chat.send_message('What products should salesperson Alice focus on to round out her portfolio? Explain why.')
print(f"\n{response.text}")

 - DB CALL: list_tables()
 - DB CALL: describe_table(products)
 - DB CALL: describe_table(staff)
 - DB CALL: describe_table(orders)
 - DB CALL: execute_query(SELECT product_id FROM orders AS t1 JOIN staff AS t2 ON t1.staff_id = t2.staff_id WHERE t2.first_name = 'Alice')
 - DB CALL: execute_query(SELECT product_id, COUNT(*) AS order_count FROM orders GROUP BY product_id ORDER BY order_count DESC LIMIT 3)

It seems no orders have been placed in the orders table. I recommend Alice focus on selling a variety of products across different price points to attract a broad customer base. Since I don't have any sales data, I can't suggest specific products.

